# Analisis CSVs Habitatnes

El Archivo Historico de Localidades es una base de datos de INEGI con registros de los diferentes censos y conteos que se han realizado en la Republica Mexicana desde el año 1900 a la fecha. 


## Librerias y configuraciones

In [1]:
import pandas as pd
import numpy as np

In [2]:
verbose = True

## Caracteristicas generales

CVE_GEOEST es el campo que comparten todas las tablas del AHL. Representa la clave geoestadistica de una localidad especifica. A veces existen problemas en su importación debido a que son solo caracteres numericos.

Esta columna debe ser importada como texto, por eso se especifica el tipo dentro del metodo read_csv. Si se importa como numero entonces se pierde el '0' al principio de algunas claves (por ejemplo '010010001' se importará como 10010001) y se perderá el formato del campo.


In [3]:
tipo_columnas = {'CVE_GEOEST': str}

# Importar archivo
habitantes = pd.read_csv('CSVs/habitantes.csv', dtype=tipo_columnas)

In [4]:
# Para procesar la sentencia y ver los resultados hay que quitar el signo #
# al principio de cada sentencia y correr la celda.

# Información general

#habitantes.memory_usage().sum() / 1_000_000  # El archivo pesa 150 megas
#habitantes.info(show_counts = True)          # No hay valores nulos en las diferentes columnas
#habitantes.describe()                        # Estadisticas por columna



CVE_GEOEST esta formada por 9 caracteres numericos. Los primeros dos representan la clave del estado ('01'), los siguientes tres caracteres representan al municipio ('001'), y los ultimos cuatro numeros representan la clave de la localidad ('0001').

El orden numerico de los Estados estan organizado por orden alfabetico. Los municipios y localidades no son consistentes con este criterio porque cuando se crean nuevos registros la numeración no es reiniciada. Solo hay una regla de nomenclatura para las localidades, la clave '0001' siempre representará la cabecera municipal.

In [5]:
habitantes.head()

,CLAVE,CVE_GEOEST,EVE_CENSAL,INDICE_HAB,FUENTE,TOT_HAB,TOT_HOM,TOT_MUJ
0,1,010010001,1940,14962331,Censo,"82,234","37,821","44,413"
1,1,010010001,1930,14962332,Censo,"62,244","28,687","33,557"
2,1,010010001,1900,14962333,Censo,"35,052","16,229","18,823"
3,1,010010001,1970,14962334,Censo,"181,277",-,-
4,1,010010001,2005,14962335,Conteo,"663,671","319,649","344,022"


La columna CLAVE tiene la misma cantidad de valores unicos que CVE_GEOEST y se pueden repetir. CLAVE tendra el mismo valor para cada CVE_GEOEST. EVE_CENSAL es un valor unico para cada uno de los registros de la tabla. 

TOT_HAB (Total de habitantes), TOT_HOM (Total de hombres), y TOT_MUJ (Total de mujeres) contienen valores numericos que representan cantidad de personas. Hay algunos campos que no tienen información numerica, por eso el tipo de información dentro de la columna esta definido como objeto y no como numero entero. 

Los valores unicos de FUENTE representan el tipo de evento censal.

In [6]:
#CVE_GEOEST y CLAVE
#len(habitantes)                                 # El CSV tiene 2138702 registros
#len(habitantes.CVE_GEOEST.unique())            # Existen 408754 claves estadisticas unicas
#habitantes.CVE_GEOEST.value_counts().max()     # La clave que mas se repite lo hace 16 veces
#habitantes.CVE_GEOEST.value_counts().min()     # La clave que menos se repite lo hace 1 vez

#set([len(clave) for clave in habitantes.CVE_GEOEST])   # Todas las claves estan compuestas por
                                                        # nueve caracteres


#len((habitantes['CVE_GEOEST'] + habitantes['CLAVE'].apply(str)).unique())
                                                # Si se concatena la columna CLAVE con CVE_GEOEST
                                                # el numero de valores unicos es el mismo (408754). 
                                                # Esto significa que el valor de clave no varia para 
                                                # cada valor de clave geoestadistica.
                
#habitantes['INDICE_HAB'].is_unique             # INDICE_HAB es un valor unico para todas las localidades
                                                # del censo. Eso significa que el valor cambia para cada 
                                                # registro.



In [7]:
lista_columnas = habitantes.columns.values.tolist()

print(f'Cantidad de registros en habitantes: {len(habitantes)}', end = '\n\n')

for col in lista_columnas[ :5]: 
    print(f'Cantidad de valores unicos de {col}: {len(habitantes[col].unique())}')
    
print()
print('Valores unicos de EVE_CENSAL:', end = '\n\n')
print(habitantes['EVE_CENSAL'].unique(), end = '\n\n')

print('Valores unicos de FUENTE:', end = '\n\n')
print(habitantes['FUENTE'].unique(), end = '\n\n')

print('Valores unicos de INDICE_HAB:', end = '\n\n')
print(len(habitantes['INDICE_HAB'].unique()), end = '\n\n')



Cantidad de registros en habitantes: 2138702

Cantidad de valores unicos de CLAVE: 408754
Cantidad de valores unicos de CVE_GEOEST: 408754
Cantidad de valores unicos de EVE_CENSAL: 19
Cantidad de valores unicos de INDICE_HAB: 2138702
Cantidad de valores unicos de FUENTE: 2

Valores unicos de EVE_CENSAL:

[1940 1930 1900 1970 2005 1995 1950 2020 1960 2010 1910 2000 1980 1990
 1921 1939 1920    2 2013]

Valores unicos de FUENTE:

['Censo' 'Conteo']

Valores unicos de INDICE_HAB:

2138702



In [8]:
# EVE_CENSAL

#sorted(habitantes.EVE_CENSAL.unique().tolist()) # Lista de valores unicos
habitantes.EVE_CENSAL.value_counts()            # Los eventos censales que menos
                                                 # registros tienen son 1939, 1920, 
                                                 # y 2013


2010    279187
2020    275867
2005    265585
2000    255718
1995    252784
1990    173771
1980    129895
1970     98154
1960     87878
1950     75029
1940     69733
1930     57411
1921     43851
1910     41045
1900     32605
1939       179
2            8
1920         1
2013         1
Name: EVE_CENSAL, dtype: int64

In [9]:
# Analisis de regirstos con EVE_CENSAL 1939, 1920, 2 ,2013

df_localidades_2 = habitantes.loc[habitantes['EVE_CENSAL'] == 2]
df_localidades_1920 = habitantes.loc[habitantes['EVE_CENSAL'] == 1920]
df_localidades_1939 = habitantes.loc[habitantes['EVE_CENSAL'] == 1939]
df_localidades_2013 = habitantes.loc[habitantes['EVE_CENSAL'] == 2013]

In [44]:
# EVE_CENSAL 2

# Todos los valores de  EVE_CENSAL representan un año en el cual se realizó un
# censo o conteo exepto el valor dos.

#df_localidades_2['EVE_CENSAL'].value_counts()      # Son claves geoestadisticas 
                                                    # que solo aparecen comunidades de censos 
                                                    # recientes
                                                    # el mas antiguo es 1990
# df_localidades_2['TOT_HAB'].dtypes                # Existen varios tipos de datos mezclados
                                                    # en la columna (numero entero y string)
#df_localidades_2['TOT_HAB'].value_counts()         # La mayor parte de los valores es 0,
                                                    # hay una comundad de 10 y otra de 6,
                                                    # hay 4 valores indefinidos.

#df_localidades_2

In [14]:
# EVE_CENSAL 1920 y 2013 solo tiene un registro con todos los campos completos. 

#df_localidades_1920  
#df_localidades_2013                # TOT_HAB, TOT_HOM, TOT_MUJ con valor 0                   

In [21]:
# EVE_CENSAL 1939

#df_localidades_1939.head()
#len(df_localidades_1939)                           # 179 registros en total

#df_localidades_1939[['TOT_HAB', 'TOT_HOM', 'TOT_MUJ']].value_counts()
                                                    # Existen 196 sin valor en ninguna columna,
                                                    # 9 con valor 0 en todas las columnas, y
                                                    # un solo registro tiene valor 2, 1 1 en
                                                    # TOT_HAB, TOT_HOM, TOT_MUJ

In [ ]:
#habitantes['TOT_HAB'] =pd.to_numeric(habitantes['TOT_HAB'], 
#                                     errors='coerce')

#habitantes['TOT_HOM'] =pd.to_numeric(habitantes['TOT_HOM'], 
#                                     errors='coerce',
#                                     downcast='integer')

#habitantes['TOT_MUJ'] =pd.to_numeric(habitantes['TOT_MUJ'], 
#                                     errors='coerce',
#                                     downcast='integer')